In [20]:
from tkinter import *
import tkinter as tk
from pathlib import Path
from datetime import datetime
from tkinter.ttk import *
import os
import re
import pandas as pd
import numpy 

'''Crating a list of all excel files in the folder'''
Direc = os.getcwd()
files = os.listdir(Direc)
files = [f for f in files if os.path.isfile(Direc+'/'+f)] #Filtering only the files.
r = re.compile(".*xlsx")
file_options = list(filter(r.match, files))

class VinkoExcel(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.geometry("400x400")
       # Label(self, text = 'Welcome to VinkoExcel!').grid()
        '''Creating frames'''
        #Creating Buttonframe
        self.buttonframe = Frame(self, width = 100, height = 50)
        self.buttonframe.grid()
        
        self.launch = Button(self.buttonframe, text = 'Launch', command = self.Launch)
        self.select_file = Button(self.buttonframe, text = 'Select file', command = self.SelectFile)
        self.select_sheet = Button(self.buttonframe, text = 'Select sheet', command = self.SelectSheet)
        self.clear_table = Button(self.buttonframe, text = 'Clear table', command = self.clear_treeview)
        self.swap_columns = Button(self.buttonframe, text = 'Swap Columns', command = self.swap_columns)
        self.execute_swap_columns = Button(self.buttonframe, text = 'Execute Swap Columns', command = self.execute_swaping_columns)
        #self.scrollbar = Scrollbar(self)
        self.option_box = Listbox(self.buttonframe)
        self.sheets_box = Listbox(self.buttonframe)
        self.swap_first_column = Listbox(self.buttonframe)
        self.swap_second_column = Listbox(self.buttonframe, exportselection=False)
        
        
        self.launch.grid(row = 1)
        
        
    def clear_treeview(self):
        self.tree.delete(*self.tree.get_children())
    
    def Launch(self):
        '''starting the program and creating selectfile button and the option box'''
        #self.option_box.delete(0,'end') # removing everything  in file options
        self.select_file.grid(row = 1)
        Label(self.buttonframe,text = 'Chose file.').grid(row = 5, column = 0)
        self.option_box.grid(row = 6, column = 0)
        self.launch.grid_forget()
        for file in range(len(file_options)):
            self.option_box.insert('end', file_options[file])
        self.openTableWindow()
    
    def SelectFile(self):
        '''selecting the working file and then creating the selectsheet button and the sheet option box'''
        self.sheets_box.delete(0,'end')
        Label(self.buttonframe,text = 'Chose sheet.').grid(row = 5, column = 1)
        #self.show.config(text=self.option_box.get(ANCHOR))
        self.chosen_file = self.option_box.get(ANCHOR)
        xl = pd.ExcelFile(self.chosen_file)  #picking sheets
        sheets_list = xl.sheet_names
        self.sheets_box.grid(row = 6, column = 1)
        self.select_sheet.grid(row = 1, column = 1)
        for sheet in range(len(sheets_list)):
            self.sheets_box.insert('end',sheets_list[sheet])
        
    def SelectSheet(self):
        '''selecting the sheet we want to work with'''
        self.chosen_sheet = self.sheets_box.get(ANCHOR)
        self.df = pd.read_excel(self.chosen_file, sheet_name = self.chosen_sheet)
        self.loading_table()
        self.clear_table.grid(row = 1, column = 3)
        self.swap_columns.grid(row = 1, column = 4)
        
    
    def swap_columns(self):
        Label(self.buttonframe,text = 'Chose first column to swap!').grid(row = 5, column = 4)
        Label(self.buttonframe,text = 'Chose second column to swap!').grid(row = 5, column = 5)
        self.swap_first_column.grid(row =6, column = 4)
        self.swap_second_column.grid(row =6, column = 5)
        for col in self.tree["column"]:
            self.swap_first_column.insert('end', col)
        for col in self.tree["column"]:
            self.swap_second_column.insert('end', col)
        self.execute_swap_columns.grid(row = 1, column = 5)
        self.first_column_swap = self.swap_first_column.get(ANCHOR)
        self.second_column_swap = self.swap_second_column.get(ANCHOR)
            
    def execute_swaping_columns(self):
        self.first_column_swap = self.swap_first_column.get(ANCHOR)
        self.second_column_swap = self.swap_second_column.get(ANCHOR)
        col_list = list(self.df.columns)
        x, y = col_list.index(self.first_column_swap), col_list.index(self.second_column_swap)
        col_list[y], col_list[x] = col_list[x], col_list[y]
        self.df = self.df[col_list]
        self.loading_table()
        
    
    def loading_table(self):
        '''filling the treeview with data from the df we chose'''
        self.tree["column"] = list(self.df.columns)
        self.tree["show"] = "headings"
        # For Headings iterate over the columns
        for col in self.tree["column"]:
            self.tree.heading(col, text=col)
            self.tree.column(col, stretch=True, width = 50)
        # Put Data in Rows
        self.df_rows = self.df.to_numpy().tolist()
        for row in self.df_rows:
            self.tree.insert("", "end", values=row)
        self.tree.pack(fill = BOTH)
        
        #Creating a scrollbar
        self.vertical_frame.config(command = self.tree.yview)
        self.vertical_frame.pack(side = RIGHT, fill = Y)
        self.tree['yscrollcommand'] = self.vertical_frame
        
        self.horizontal_frame.config(command = self.tree.xview)
        self.horizontal_frame.pack(side = BOTTOM, fill = X)
        self.tree['xscrollcommand'] = self.horizontal_frame
        
    def openTableWindow(self):
        self.tableWindow = Toplevel(self)
        self.tableWindow.geometry('500x300')
        self.tableWindow.title('Table Window')
        self.vertical_frame = Scrollbar(self.tableWindow, orient="vertical")
        self.horizontal_frame = Scrollbar(self.tableWindow, orient="horizontal")
        
        #Creating treeview  '''h = 20,'''
        self.tree = Treeview(self.tableWindow, xscrollcommand=self.horizontal_frame.set, yscrollcommand=self.vertical_frame.set)
        #clear_treeview()



app = VinkoExcel()
app.mainloop()